# Gyges Playground

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json
import time

DATA_FOLDER = Path("../data")
BOARD_SIZE = 6

## Game Setup
we need a **Board**, which is a collection of **Cells**, and a **Game** object with the game logic

In [2]:
class GygesBoard:
    def __init__(self, size):
        self.size = size
        self.board = np.array([
            [Cell(x, y) for x in range(size)]
            for y in range(size + 2)
        ])
        # Set the home rows
        for j in range(size):
            self[0, j].is_white_home = True
            self[-1, j].is_black_home = True

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.board[idx]

    def __setitem__(self, idx, value):
        self.board[idx].value = value

    def __iter__(self):
        return iter(self.board)

    def __repr__(self):
        string = ""
        for j in range(self.size + 2):
            if j == 0:
                j_idx = 'W'
            elif j == self.size + 1:
                j_idx = 'B'
            else:
                j_idx = j
            string += f"{j_idx}| {' '.join(f'{str(item):^3}' for item in self.board[j])} |\n"
        
        string += f"   {' '.join(f'{str(n):^3}' for n in range(self.size))} \n"
        return string


class Cell:
    def __init__(self, x=0, y=0, value=0):
        self.x = x
        self.y = y
        self.value = value
        self.visited = False
        self.is_white_home = False
        self.is_black_home = False

    def __add__(self, other):
        if isinstance(other, Cell):
            return Cell(value = self.value + other.value)
        elif isinstance(other, int) or isinstance(other, float):
            return Cell(value = self.value + other)
        else:
            raise ValueError("Can add only Cells, Integers or Floats")
        
    def __eq__(self, value):
        return self.value == value

    def __repr__(self):
        if self.is_white_home or self.is_black_home:
            return '-'
        if self.visited:
            return f"{self.value if self.value > 0 else ''}x"
        else:
            return str(self.value)

b = GygesBoard(BOARD_SIZE)
b[1, 2] = 1
b

W|  -   -   -   -   -   -  |
1|  0   0   1   0   0   0  |
2|  0   0   0   0   0   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  0   0   0   0   0   0  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  

In [ ]:
class GygesGame:
    def __init__(self, white_pieces, black_pieces):
        self.gameboard = GygesBoard(BOARD_SIZE)

        self.player = 1
        self.over = False
        self.winning_player = None
        
        # White pieces are in row 0
        w_starting_row = self.closest_white_row
        b_starting_row = self.closest_black_row
        for i, piece in enumerate(white_pieces):
            self.gameboard[w_starting_row, i] = piece
        # Black pieces are in row -1
        for i, piece in enumerate(black_pieces):
            self.gameboard[b_starting_row, i] = piece

    @property
    def player_to_color(self):
        return {0: "B", 1: "W"}
    
    @property
    def active_row(self):
        if self.player == 1:
            return self.closest_white_row
        elif self.player == 0:
            return self.closest_black_row

    @property
    def closest_white_row(self):
        # If no piece is set
        if np.sum(self.gameboard) == 0:
            return 1  # Starting white row
        for i, value in enumerate(np.sum(self.gameboard, axis=1)):
            if value != 0:
                return i
        
    @property
    def closest_black_row(self):
        # If no piece is set
        if np.sum(self.gameboard) == 0:
            return -2  # Starting black row
        # Reverse the list
        for i, value in enumerate(np.sum(self.gameboard, axis=1)[::-1]):
            if value != 0:
                return - (i + 1)
            
    @property
    def status_string(self):
        if self.over:
            return f"PLAYER {self.player_to_color[self.winning_player]} WINS!\n"
        else:
            return f"Player {self.player_to_color[self.player]} turn.\n"
            
    def clean_board(self):
        for row in self.gameboard:
            for cell in row:
                cell.visited = False

        return self
            
    def select_cell(self, y, x):
        if self.gameboard[y, x] == 0:
            raise ValueError("Invalid Selection!\nNo Piece in the selected Cell")
        
        if self.player == 0 and y != self.closest_black_row:
            raise ValueError("Invalid Selection!\nSelected cell must be in the closest black row")
        if self.player == 1 and y != self.closest_white_row:
            raise ValueError("Invalid Selection!\nSelected cell must be in the closest black row")
        y_in_range = y != 0 and y != -1 and y != self.gameboard.size
        x_in_range = 0 <= x <= self.gameboard.size
        if y_in_range and x_in_range:
            self.clean_board()
            self.selected_cell = self.gameboard[y, x]
            self.selected_piece = self.gameboard[y, x].value
            # Selected Piece is ready to be moved and cell must be emptied
            self.gameboard[y, x] = 0
            return self.selected_piece
        else:
            raise ValueError("Invalid Selection!\nCell must be within boundaries")
        
    @staticmethod
    def _check_valid_move(dy, dx):
        if dx != 0 and dy != 0:
            raise ValueError("Invalid Move!\nYou can move only on x or y")
        if dx == 0 and dy == 0:
            raise ValueError("Invalid Move!\nYou must move x or y")
        if dx == 0:
            if dy not in [1, -1]:
                raise ValueError("Invalid Move!\nYou can move only by 1 or -1")
        if dy == 0:
            if dx not in [1, -1]:
                raise ValueError("Invalid Move!\nYou can move only by 1 or -1")

    def _check_winning_conditions(self, new_cell):
        if self.player == 1: 
            if new_cell.is_black_home:
                self.winning_player = self.player
                self.over = True
                return True
            elif new_cell.is_white_home:
                raise ValueError("Invalid Move!\nYou cannot finish in your own home row")
        elif self.player == 0:
            if new_cell.is_white_home:
                self.winning_player = self.player
                self.over = True
                return True
            elif new_cell.is_black_home:
                raise ValueError("Invalid Move!\nYou cannot finish in your own home row")
        else:
            return False
    
    def slide(self, path, visualize=False):
        current_cell = self.selected_cell
        for i, (dy, dx) in enumerate(path):
            y, x = current_cell.y, current_cell.x
            self.gameboard[y, x].visited = True
            new_y = int(y + dy)
            new_x = int(x + dx)
            if new_x < 0 or new_x >= self.gameboard.size:
                raise ValueError("Invalid Move!\nYou cannot move across borders")
            new_cell = self.gameboard[new_y, new_x]
            message = f"Piece {self.selected_piece} moved from cell {(y, x)} to cell: {(new_y, new_x)}"
            
            # Check winning conditions and returns only at the last step
            if i == len(path) - 1:
                if self._check_winning_conditions(new_cell):
                    return 0  # No more moves, game ends
                else:
                    # The game goes on
                    self.selected_cell = new_cell
                    # print(message)
                    if visualize:
                        print(self)
                    return new_cell.value
            else:
                # Prevent invalid passages on occupied cells or home rows
                if new_cell.is_black_home or new_cell.is_white_home:
                    raise ValueError("Invalid Move!\nYou cannot pass through the home row")
                if new_cell.value:
                    raise ValueError("Invalid Move!\nYou cannot pass through an occupied Cell")
            # print(message)
            current_cell = new_cell
            if visualize:
                print(self)

    def place_piece(self):
        y, x = self.selected_cell.y, self.selected_cell.x
        self.gameboard[y, x] = self.selected_piece
    
    def __repr__(self):
        string = str(self.gameboard)
        string += self.status_string
        return string
        
w_starting_config = [2, 1, 3, 2, 3, 1]
b_starting_config = [2, 1, 3, 3, 1, 2]
game = GygesGame(w_starting_config, b_starting_config)
game

W|  -   -   -   -   -   -  |
1|  2   1   3   2   3   1  |
2|  0   0   0   0   0   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

## Simulate a Game

In [4]:
correct_moves = [
    (1, 0), (-1, 0), (0, -1), (0, 1)
]   

rng = np.random.default_rng(seed=42)
game = GygesGame(w_starting_config, b_starting_config)
strategy = {0: [(1, 2), [(1, 0), (0, 1), (-1, 0)]]}
current_turn = 0
current_strategy = strategy[current_turn]

game.select_cell(*current_strategy[0])
jump = game.selected_piece
next_strategy = current_strategy[1]
print(next_strategy)
while jump > 0:
    jump = game.slide(next_strategy, visualize=True)
    next_strategy = rng.choice(correct_moves, jump)
# Place the Piece in the selected Cell
game.place_piece()

game

[(1, 0), (0, 1), (-1, 0)]
W|  -   -   -   -   -   -  |
1|  2   1   x   2   3   1  |
2|  0   0   0   0   0   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

W|  -   -   -   -   -   -  |
1|  2   1   x   2   3   1  |
2|  0   0   x   0   0   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

Piece 3 moved from cell (2, 3) to cell: (1, 3)
W|  -   -   -   -   -   -  |
1|  2   1   x   2   3   1  |
2|  0   0   x   x   0   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

W|  -   -   -   -   -   -  |
1|  2   1   x  2x   3   1  |
2|  0   0   x   x   0   0  |
3|  0   0   

W|  -   -   -   -   -   -  |
1|  2   1   x  2x   3   1  |
2|  0   0   x   x   3   0  |
3|  0   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

In [4]:
# game = GygesGame(w_starting_config, b_starting_config)

def make_complete_movement(game, cell, strategies=[], seed=42):
    if isinstance(seed, np.random.Generator):
        rng = seed
    else:
        rng = np.random.default_rng(seed=seed)
    correct_moves = [
        (1, 0), (-1, 0), (0, -1), (0, 1)
    ]
    game.select_cell(*cell)
    jump = game.selected_piece
    i = 0
    jump = 2
    while jump > 0:
            random_strategy = rng.choice(correct_moves, jump, replace=True)
            if len(strategies) - 1 < i:
                strategies.append(random_strategy)
            elif strategies[i] is None:
                # Generate a random value
                strategies[i] = random_strategy
            
            strategy = strategies[i]

            try:
                jump = game.slide(strategy, visualize=False)
            except ValueError:
                strategies[i] = None
            # When a valid strategy is found, overwrite the value
            strategies[i] = strategy
            i += 1

    game.place_piece()
    return strategies

# print(game)
# make_complete_movement(game, (1, 0), seed=124214)
# print(game)

In [ ]:
strategy = {0: [(1, 2), [(1, 0), (0, -1), (-1, 0), (1, 0)]]}
current_turn = 0
current_strategy = strategy[current_turn]
available_moves = game.select_cell(*current_strategy[0])
m = 0
while available_moves > 0:
    piece_movements = current_strategy[1]
    if game.over:
        break
    dydx = piece_movements[m]
    m += 1
    available_moves = game.move(available_moves, dydx)
    print(game)
game.clean_board()

with open(DATA_FOLDER / f"strategy1.json", "w") as f:
    json.dump(strategy, f)

W|  -   -   -   -   -   -  |
1|  0   1   x   2   3   1  |
2|  0   0   0   0   0   0  |
3|  2   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

W|  -   -   -   -   -   -  |
1|  0   1   x   2   3   1  |
2|  0   0   x   0   0   0  |
3|  2   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

W|  -   -   -   -   -   -  |
1|  0   1   x   2   3   1  |
2|  0   x   x   0   0   0  |
3|  2   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  |
6|  2   1   3   3   1   2  |
B|  -   -   -   -   -   -  |
    0   1   2   3   4   5  
Player W turn.

W|  -   -   -   -   -   -  |
1|  0  1x   x   2   3   1  |
2|  0  3x   x   0   0   0  |
3|  2   0   0   0   0   0  |
4|  0   0   0   0   0   0  |
5|  0   0   0   0   0   0  

# Random Playthrough

In [ ]:
correct_moves = [
    (1, 0), (-1, 0), (0, -1), (0, 1)
]
def move_a_piece(available_moves, correct_moves):
    t = 0
    correct_moves
    while t < 4:
        dydx = rng.choice(correct_moves)
        try:
            available_moves = game.move(available_moves, dydx)
        except ValueError:
            t += 1
            correct_moves.pop(np.where(correct_moves == dydx))
        else:
            # Exit the loop
            return dydx, available_moves
    # If it exits here it exhausted all the possible moves and need to rethink all the strategy
    

game = GygesGame(w_starting_config, b_starting_config)
rng = np.random.default_rng(seed=42)

In [5]:
class Player:
    def __init__(self, name, mind='random', seed=42, verbose=False):
        self.rng = np.random.default_rng(seed=seed)
        self.name = name
        self.mind = mind
        self.is_playing = False
        self.game = None
        self.verbose = verbose
    
    def play(self, game):
        self.is_playing = True
        self.game = game

    def seed(self, seed):
        self.rng = np.random.default_rng(seed=seed)

    def choose_piece(self):
        active_row = self.game.active_row
        cells_with_pieces = [
            i for i in range(self.game.gameboard.size) 
            if game.gameboard[active_row, i].value > 0]

        if self.mind == 'random':
            cell = int(rng.choice(cells_with_pieces))
        elif self.mind == 'human':
            while True:
                cell = input(f"You can choose a Piece in row {active_row} ({cells_with_pieces}): ")
                if cell[0] != active_row:
                    print("Bad row selection")
                elif cell[1] not in cells_with_pieces:
                    print("Bad column selection")
                else:
                    break
        cell = (active_row, cell)
        if self.verbose:
            print(f"{self.name}: select cell {cell}.")
        return cell
    
    def __str__(self):
        return str(self.name)


In [6]:
rng = np.random.default_rng(seed=142)
for seed in rng.integers(1, 999999, 1000):
    game = GygesGame(w_starting_config, b_starting_config)
    rng = np.random.default_rng(seed=seed)

    strategy = {}
    num_to_color = {0: "B", 1: "W"}
    player0 = Player("B", verbose=False)
    player1 = Player("W", verbose=False)
    num_to_player = {0: player0, 1: player1}


    player0.play(game)
    player1.play(game)

    game.player = 1  # White starts
    t = 0
    max_turns = 10000
    while not game.over and t < max_turns:
        turn_moves = []
        active_player = num_to_player[game.player]
        selected_cell = active_player.choose_piece()

        movements = make_complete_movement(game, selected_cell, seed=rng)

        # The strategy is a collection of all the moves for a selected cell in a turn
        strategy[t] = [selected_cell, [m.tolist() for m in movements]]
        t += 1
        game.player = (game.player + 1) % 2 

    with open(DATA_FOLDER / f"strategy_seed{seed}.json", "w") as f:
        json.dump(strategy, f)

KeyboardInterrupt: 

In [ ]:
[m.tolist() for m in strategy[0][1]]

In [ ]:
print(game)
print(game.closest_white_row)

game.gameboard[0, 4].is_white_home

In [ ]:
for i in np.sum(b.board, axis=1)[::-1]:
    if i != 0:
        break

print(i)